In [2]:
from scipy.stats import binom
from scipy.stats import geom
from scipy.stats import hypergeom
from scipy.stats import poisson
from scipy.stats import nbinom
from scipy.stats import expon
from scipy.stats import norm
from scipy import stats
import pandas as pd
import sympy as sp
from sympy import *
sp.init_printing()
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from scipy.integrate import quad
import numpy as np
from scipy.misc import derivative
import math
from math import *
from fractions import Fraction
import fractions
from IPython.display import display, Math, Latex
from sympy.abc import x
from sympy import Poly
from sympy.solvers.inequalities import solve_rational_inequalities
%matplotlib inline

# Assignment 1

A company purchases tires from two different suppliers, 1 and 2. Records show that 10% of the tires from supplier 1 and 5% of the tires from supplier 2 are defective. 40% of the company’s current tire inventory stems from supplier 1, and the remaining from supplier 2. The company has a total inventory of 1000 tires. 

a. Based on this information, construct a contingency table of the company’s tire inventory situation (2x2 contingency table). 


In [3]:
#the table
data = [["Good",360,570,930 ],["Defective", 40,30,70],["Total", 400 ,600,1000]]
pd.DataFrame(data , columns=["Condition","Supplier 1", "Supplier 2","Total"])

,Condition,Supplier 1,Supplier 2,Total
0,Good,360,570,930
1,Defective,40,30,70
2,Total,400,600,1000


b. If a randomly chosen tires from the company’s inventory is chosen and turns out to be defective, what is the probability that the tire is from supplier 1? 

In [5]:
#Probability of supplier 1, GIVEN that the tire is defective: 
# P(Supplier 1 | Defective)
p = 40/70
p

c. Is there sufficient evidence to support the claim that the rate of defectives depends very significantly on supplier?

In [9]:
# H0 = The chances of getting a defective tire is independent of the supplier.
# H1 = The chances of getting a defective tire is dependent of the supplier.

# calc the expected values
data = [["Good",400*930/1000,600*930/1000,930 ],["Defective", 400*70/1000,600*70/1000,70],["Total", 400 ,600,1000]]
pd.DataFrame(data , columns=["Condition","Supplier 1", "Supplier 2","Total"])

,Condition,Supplier 1,Supplier 2,Total
0,Good,372.0,558.0,930
1,Defective,28.0,42.0,70
2,Total,400.0,600.0,1000


In [8]:
# clculte x pow 2
X2 = (360-372)**2/372+(570-558)**2/558+(40-28)**2/28+(30-42)**2/42
print('X2 = ', X2)

# the Degrees of freedom is calculated: Dof = (rows-1)*(columns-1)
dof = (2-1)*(2-1)
print('Degrees of freedom = ', dof)
print('P_value = ',stats.chi2.sf(X2, dof))

# can also be calculated in the following way:
obs = [[360,570],[40,30]]
chi2_stat, p_val, dof, ex2 = stats.chi2_contingency(obs, correction=False)
print('P_value = ',p_val)

X2 =  9.216589861751153
Degrees of freedom =  1
P_value =  0.0023983184964129424
P_value =  0.0023983184964129424


As the P-value is smaller than the significance value (0.0024<0.01), we can reject the null hypothesis. 

The chance of getting a defective tire does depend on the supplier. 

# Assignment 2

A SMP-teacher believes that the average height in his SMP class is normal distributed with an average of 1.7 meter and with a standard deviation of 10 cm. 
He then measure 5 students and get the following results: 

In [4]:
Height = np.array([1.83, 1.96, 1.76, 1.78, 1.68])

1. What is the sample mean? 

In [5]:
mean = Height.mean()
mean

2. What is the sample variance? 

In [7]:
#Two ways of doing it:
variance = np.sum((Height-Height.mean())**2)/(5-1)
variance2 = Height.var(ddof=1)
print(variance)
print(variance2)

0.01072
0.01072


3. What is the sample standard deviation? 

In [28]:
stdev = sqrt(variance)
print(stdev)
stdev2 = Height.std(ddof=1)
print(stdev2)

0.10353743284435828
0.10353743284435828


4. What is the P-value (2-tailed)? 

In [29]:
#This is the formula to do it without calculate z value:
#the scale is the standard deviation, we are using meters so we convert 10 cm in meters (0.1)
#loc is the mean/average/normally distributed around..
#5 is the amount of students/sample amount
prob = norm.cdf(Height.mean(), loc=1.7, scale=0.1/np.sqrt(5)) 
p_val = 2*min(prob, 1-prob) 
print('p_value = ', p_val) # p_value =  0.0283795

p_value =  0.022560915657808822


In [30]:
#Calculating z value first, you can use it to check.
z = (Height.mean()-1.7) / ((0.1)/sqrt(5))
prob = norm.cdf(z)
p_val = 2*min(prob, 1-prob) 
p_val

5. Can the belief of the SMP-teacher be rejected with a significant level α, of 5% ? 

In [31]:
#Yes because the P value is lower than the significant level (0.0225 < 0.05)

After rejecting the hypothesis the SMP teacher argues that the average height is 1.7m and it is normal distributed, but admits that he don’t know the true standard deviation. 

6. What is the P-value (2-tailed) when the standard deviation is unknown? 

In [32]:
#Use this when variance/standart deviation is unknown
prob = stats.t.cdf(Height.mean(), df=len(Height)-1, loc=1.7, scale=Height.std(ddof=1)/np.sqrt(len(Height))) 
p_val = 2*min(prob, 1-prob) 
print('p_value = ', p_val) #p_value =  0.092357 

p_value =  0.09235730899034578


7. Can the belief of the SMP-teacher be rejected with a significant level α, of 5% ? 

In [33]:
#No because the P value is higher than the significance level (0.09 > 0.05).  The SMP-teacher might be right.  

# Assignment 3

An urban community would like to show that the incidence of breast cancer is higher in their area than in a nearby rural area. (PCB levels were found to be higher in the soil of the urban community.) If it is found that 20 of 200 adult women in the urban community have breast cancer and 10 of 150 adult women in the rural community have breast can we conclude at the 0.05 level of significance that breast cancer is more prevalent in the urban community? 

In [34]:
# H0 : Breast cancer is independent of the living area
# H1 : Breast cancer is dependent of the living area

In [37]:
# h0 the sickness is independent of the place of living
# make the table
data = [["Women without cancer",180,140,320],["Women with cancer", 20,10,30],["Total", 200 ,150,350]]
pd.DataFrame(data , columns=["Condition","urban", "rural","Total"])

,Condition,urban,rural,Total
0,Women without cancer,180,140,320
1,Women with cancer,20,10,30
2,Total,200,150,350


In [38]:
# calc the expected values
data = [["Women without cancer",200*320/350,150*320/350,320],["Women with cancer", 200*30/350,150*30/350,30],["Total", 200 ,150,350]]
pd.DataFrame(data , columns=["Condition","urban", "rural","Total"])

,Condition,urban,rural,Total
0,Women without cancer,182.857143,137.142857,320
1,Women with cancer,17.142857,12.857143,30
2,Total,200.000000,150.000000,350


In [41]:
# clculte x pow 2
X2 = (180-182.8)**2/182.8+(140-137.2)**2/137.2+(20-17.1)**2/17.7+(10-12.9)**2/12.9
print('X2 = ', X2)

# the Degrees of freedom is calculated: Dof = (rows-1)*(columns-1)
dof = (2-1)*(2-1)
print('Degrees of freedom = ', dof)
print('P_value = ',stats.chi2.sf(X2, dof))

# can also be calculated in the following way, if you have a lot of data use this!:
obs = [[20,10],[180,140]] 
chi2_stat, p_val, dof, ex2 = stats.chi2_contingency(obs, correction=False)
print('P_value = ',p_val)

X2 =  1.2271104872026553
Degrees of freedom =  1
P_value =  0.26796971633132877
P_value =  0.2702893848016953


In [42]:
#We cannot reject the null hypothesis because the P value is higher than the significance level (0.27 > 0.05). 
#The breast cancer is independent of the living area.

# Assignment 4

Different screens and their hue bias were tested and the result is displayed in the following table: 

In [43]:
data = [["display1",46,82,72,200 ],["display2", 42,38,20,100], ["display3", 52,40,8,100], ["total",140,160,100,400]]
pd.DataFrame(data , columns=["display","blueish", "reddish","greenish", "total"])

,display,blueish,reddish,greenish,total
0,display1,46,82,72,200
1,display2,42,38,20,100
2,display3,52,40,8,100
3,total,140,160,100,400


Is there sufficient evidence to conclude that screens and hue bias depend significantly? 
Design an appropriate test to answer this question. 

In [44]:
# H0: The hue bias is independent of the screen
# H1: The hue bias is dependent of the screen

In [46]:
obs = [[46,82,72],[42,38,20], [52,40,8]] 
chi2_stat, p_val, dof, ex2 = stats.chi2_contingency(obs, correction=False)
print('Expected values = \n',ex) # Expected values =  [[70. 80. 50.] [35. 40. 25.] [35. 40. 25.]] 
print('Degrees of freedom = ',dof) # Degrees of freedom =  4 
print('chi2 = ',chi2)   # chi2 =  40.2757142 print
print('P_value = ',p_val)

Expected values = 
 [[ 17.14285714  12.85714286]
 [182.85714286 137.14285714]]
Degrees of freedom =  4
chi2 =  1.2152777777777786
P_value =  3.795777297500132e-08


In [ ]:
#As the P-value is lower than the significance level (let’s say 0.05) the null-hypothesis can be rejected. 
#We reject the null hypothesis because p value is lower than 0.05. The hue bias depends of the screen.